In [1]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [3]:
import threading

In [4]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [5]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [6]:
import datetime
import os

In [7]:
import pandas as pd
import h5py

In [8]:
# Setup data seed
np.random.seed(3254)

In [9]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [10]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)

#     c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
#     c4 = BatchNormalization(axis=1)(c4)
#     c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
#     c4 = BatchNormalization(axis=1)(c4)
#     p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

#     c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
#     c5 = BatchNormalization(axis=1)(c5)
#     c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
#     c5 = BatchNormalization(axis=1)(c5)

#     u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
#     u6 = concatenate([u6, c4])
#     c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
#     c6 = BatchNormalization(axis=1)(c6)
#     c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)
#     c6 = BatchNormalization(axis=1)(c6)

#     u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
#     u7 = concatenate([u7, c3])
#     c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
#     c7 = BatchNormalization(axis=1)(c7)
#     c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
#     c7 = BatchNormalization(axis=1)(c7)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c4)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = BatchNormalization(axis=1)(c7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
    c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [12]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/depthtest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., callbacks=[<keras.ca..., epochs=50, steps_per_epoch=100, verbose=1)`


Epoch 1/50
100/100 [==============================] - 18s 180ms/step - loss: 4.8871 - binary_crossentropy: 0.4980 - jaccard_coef_int: 0.1780 - binary_accuracy: 0.8218 - val_loss: 9.1485 - val_binary_crossentropy: 1.4958 - val_jaccard_coef_int: 5.2531e-04 - val_binary_accuracy: 0.7979
Epoch 2/50
100/100 [==============================] - 12s 115ms/step - loss: 4.3193 - binary_crossentropy: 0.6652 - jaccard_coef_int: 0.2202 - binary_accuracy: 0.8319 - val_loss: 9.6856 - val_binary_crossentropy: 1.3089 - val_jaccard_coef_int: 8.9234e-16 - val_binary_accuracy: 0.7994
Epoch 3/50
100/100 [==============================] - 12s 116ms/step - loss: 4.2155 - binary_crossentropy: 0.7158 - jaccard_coef_int: 0.2150 - binary_accuracy: 0.8386 - val_loss: 8.3913 - val_binary_crossentropy: 1.3753 - val_jaccard_coef_int: 0.0053 - val_binary_accuracy: 0.7846
Epoch 4/50
100/100 [==============================] - 11s 113ms/step - loss: 4.1722 - binary_crossentropy: 0.7030 - jaccard_coef_int: 0.2415 - binary

Epoch 7/50
100/100 [==============================] - 9s 90ms/step - loss: 2.4539 - binary_crossentropy: 0.0783 - jaccard_coef_int: 0.1159 - binary_accuracy: 0.9895 - val_loss: 2.9745 - val_binary_crossentropy: 0.1250 - val_jaccard_coef_int: 0.0660 - val_binary_accuracy: 0.9848
Epoch 8/50
100/100 [==============================] - 9s 91ms/step - loss: 2.4045 - binary_crossentropy: 0.0830 - jaccard_coef_int: 0.1183 - binary_accuracy: 0.9899 - val_loss: 3.0998 - val_binary_crossentropy: 0.0990 - val_jaccard_coef_int: 0.0551 - val_binary_accuracy: 0.9871
Epoch 9/50
100/100 [==============================] - 9s 94ms/step - loss: 2.4162 - binary_crossentropy: 0.0926 - jaccard_coef_int: 0.1184 - binary_accuracy: 0.9894 - val_loss: 2.7434 - val_binary_crossentropy: 0.1206 - val_jaccard_coef_int: 0.0822 - val_binary_accuracy: 0.9855
Epoch 10/50
100/100 [==============================] - 9s 91ms/step - loss: 2.3302 - binary_crossentropy: 0.0932 - jaccard_coef_int: 0.1294 - binary_accuracy: 0.98

Epoch 13/50
100/100 [==============================] - 9s 91ms/step - loss: 3.1549 - binary_crossentropy: 0.1096 - jaccard_coef_int: 0.0671 - binary_accuracy: 0.9802 - val_loss: 8.4905 - val_binary_crossentropy: 0.0696 - val_jaccard_coef_int: 2.4163e-14 - val_binary_accuracy: 0.9919
Epoch 14/50
100/100 [==============================] - 9s 90ms/step - loss: 3.1099 - binary_crossentropy: 0.1159 - jaccard_coef_int: 0.0705 - binary_accuracy: 0.9807 - val_loss: 4.2487 - val_binary_crossentropy: 0.1776 - val_jaccard_coef_int: 0.0229 - val_binary_accuracy: 0.9706
Epoch 15/50
100/100 [==============================] - 9s 91ms/step - loss: 3.2988 - binary_crossentropy: 0.1362 - jaccard_coef_int: 0.0652 - binary_accuracy: 0.9768 - val_loss: 9.4304 - val_binary_crossentropy: 0.0773 - val_jaccard_coef_int: 2.2889e-14 - val_binary_accuracy: 0.9915
Epoch 16/50
100/100 [==============================] - 9s 90ms/step - loss: 3.1665 - binary_crossentropy: 0.1212 - jaccard_coef_int: 0.0728 - binary_acc

...

Epoch 29/50
100/100 [==============================] - 9s 93ms/step - loss: 1.7855 - binary_crossentropy: 0.2330 - jaccard_coef_int: 0.2477 - binary_accuracy: 0.9768 - val_loss: 3.3834 - val_binary_crossentropy: 0.1377 - val_jaccard_coef_int: 0.0421 - val_binary_accuracy: 0.9860
Epoch 30/50
100/100 [==============================] - 9s 92ms/step - loss: 1.7317 - binary_crossentropy: 0.2272 - jaccard_coef_int: 0.2570 - binary_accuracy: 0.9779 - val_loss: 2.6493 - val_binary_crossentropy: 0.1450 - val_jaccard_coef_int: 0.0895 - val_binary_accuracy: 0.9855
Epoch 31/50
100/100 [==============================] - 9s 93ms/step - loss: 1.6669 - binary_crossentropy: 0.2237 - jaccard_coef_int: 0.2722 - binary_accuracy: 0.9783 - val_loss: 2.6734 - val_binary_crossentropy: 0.2181 - val_jaccard_coef_int: 0.0985 - val_binary_accuracy: 0.9786
Epoch 32/50
100/100 [==============================] - 9s 93ms/step - loss: 1.6865 - binary_crossentropy: 0.2205 - jaccard_coef_int: 0.2655 - binary_accuracy: 0

Epoch 6/50
100/100 [==============================] - 12s 118ms/step - loss: 3.0121 - binary_crossentropy: 0.1773 - jaccard_coef_int: 0.0969 - binary_accuracy: 0.9655 - val_loss: 3.2295 - val_binary_crossentropy: 0.1744 - val_jaccard_coef_int: 0.0718 - val_binary_accuracy: 0.9668
Epoch 7/50
100/100 [==============================] - 12s 116ms/step - loss: 2.9068 - binary_crossentropy: 0.1904 - jaccard_coef_int: 0.1039 - binary_accuracy: 0.9653 - val_loss: 3.4022 - val_binary_crossentropy: 0.1456 - val_jaccard_coef_int: 0.0528 - val_binary_accuracy: 0.9752
Epoch 8/50
100/100 [==============================] - 12s 117ms/step - loss: 2.8228 - binary_crossentropy: 0.1940 - jaccard_coef_int: 0.1101 - binary_accuracy: 0.9661 - val_loss: 3.1877 - val_binary_crossentropy: 0.1699 - val_jaccard_coef_int: 0.0711 - val_binary_accuracy: 0.9699
Epoch 9/50
100/100 [==============================] - 12s 117ms/step - loss: 2.6905 - binary_crossentropy: 0.1879 - jaccard_coef_int: 0.1188 - binary_accurac

Epoch 12/50
100/100 [==============================] - 9s 94ms/step - loss: 3.4692 - binary_crossentropy: 0.5701 - jaccard_coef_int: 0.1143 - binary_accuracy: 0.9208 - val_loss: 4.5599 - val_binary_crossentropy: 0.4793 - val_jaccard_coef_int: 0.0283 - val_binary_accuracy: 0.9376
Epoch 13/50
100/100 [==============================] - 9s 92ms/step - loss: 3.4334 - binary_crossentropy: 0.5750 - jaccard_coef_int: 0.1171 - binary_accuracy: 0.9219 - val_loss: 4.6041 - val_binary_crossentropy: 0.9875 - val_jaccard_coef_int: 0.0760 - val_binary_accuracy: 0.8741
Epoch 14/50
100/100 [==============================] - 9s 92ms/step - loss: 3.3501 - binary_crossentropy: 0.5792 - jaccard_coef_int: 0.1237 - binary_accuracy: 0.9235 - val_loss: 4.1615 - val_binary_crossentropy: 1.0362 - val_jaccard_coef_int: 0.1126 - val_binary_accuracy: 0.8772
Epoch 15/50
100/100 [==============================] - 10s 97ms/step - loss: 3.3274 - binary_crossentropy: 0.5899 - jaccard_coef_int: 0.1260 - binary_accuracy: 

Epoch 18/50
100/100 [==============================] - 9s 94ms/step - loss: 2.9778 - binary_crossentropy: 0.5677 - jaccard_coef_int: 0.1454 - binary_accuracy: 0.9352 - val_loss: 4.2001 - val_binary_crossentropy: 0.6597 - val_jaccard_coef_int: 0.0485 - val_binary_accuracy: 0.9264
Epoch 19/50
100/100 [==============================] - 9s 93ms/step - loss: 2.9319 - binary_crossentropy: 0.5713 - jaccard_coef_int: 0.1507 - binary_accuracy: 0.9360 - val_loss: 4.0994 - val_binary_crossentropy: 0.7108 - val_jaccard_coef_int: 0.0593 - val_binary_accuracy: 0.9202
Epoch 20/50
100/100 [==============================] - 9s 93ms/step - loss: 2.8586 - binary_crossentropy: 0.5673 - jaccard_coef_int: 0.1573 - binary_accuracy: 0.9379 - val_loss: 3.9835 - val_binary_crossentropy: 0.6832 - val_jaccard_coef_int: 0.0635 - val_binary_accuracy: 0.9230
Epoch 21/50
100/100 [==============================] - 9s 92ms/step - loss: 2.7935 - binary_crossentropy: 0.5717 - jaccard_coef_int: 0.1658 - binary_accuracy: 0

Epoch 24/50
100/100 [==============================] - 12s 116ms/step - loss: 2.5664 - binary_crossentropy: 0.1103 - jaccard_coef_int: 0.1386 - binary_accuracy: 0.9871 - val_loss: 5.4892 - val_binary_crossentropy: 0.0772 - val_jaccard_coef_int: 0.0048 - val_binary_accuracy: 0.9921
Epoch 25/50
100/100 [==============================] - 12s 116ms/step - loss: 2.4467 - binary_crossentropy: 0.1187 - jaccard_coef_int: 0.1559 - binary_accuracy: 0.9864 - val_loss: 7.1434 - val_binary_crossentropy: 1.1260 - val_jaccard_coef_int: 0.0042 - val_binary_accuracy: 0.8998
Epoch 26/50
100/100 [==============================] - 12s 116ms/step - loss: 2.9334 - binary_crossentropy: 0.1537 - jaccard_coef_int: 0.1232 - binary_accuracy: 0.9825 - val_loss: 9.8132 - val_binary_crossentropy: 0.0056 - val_jaccard_coef_int: 0.6250 - val_binary_accuracy: 0.9994
Epoch 27/50
100/100 [==============================] - 12s 119ms/step - loss: 2.4872 - binary_crossentropy: 0.1122 - jaccard_coef_int: 0.1520 - binary_acc

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/depthtest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., callbacks=[<keras.ca..., epochs=50, steps_per_epoch=100, verbose=1)`


Epoch 1/50
100/100 [==============================] - 38s 382ms/step - loss: 5.8234 - binary_crossentropy: 0.5745 - jaccard_coef_int: 0.3056 - binary_accuracy: 0.7609 - val_loss: 7.8147 - val_binary_crossentropy: 0.9793 - val_jaccard_coef_int: 0.0025 - val_binary_accuracy: 0.7863
Epoch 2/50
100/100 [==============================] - 9s 93ms/step - loss: 4.7966 - binary_crossentropy: 0.6986 - jaccard_coef_int: 0.3869 - binary_accuracy: 0.7797 - val_loss: 5.3779 - val_binary_crossentropy: 1.0094 - val_jaccard_coef_int: 0.3519 - val_binary_accuracy: 0.7362
Epoch 3/50
100/100 [==============================] - 9s 93ms/step - loss: 4.5514 - binary_crossentropy: 0.8241 - jaccard_coef_int: 0.3920 - binary_accuracy: 0.7838 - val_loss: 9.7310 - val_binary_crossentropy: 1.3521 - val_jaccard_coef_int: 8.4668e-16 - val_binary_accuracy: 0.7873
Epoch 4/50
100/100 [==============================] - 9s 93ms/step - loss: 4.3437 - binary_crossentropy: 0.8929 - jaccard_coef_int: 0.4050 - binary_accuracy:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100/100 [==============================] - 9s 93ms/step - loss: 1.6891 - binary_crossentropy: 0.1337 - jaccard_coef_int: 0.2315 - binary_accuracy: 0.9888 - val_loss: 2.6849 - val_binary_crossentropy: 0.2405 - val_jaccard_coef_int: 0.0951 - val_binary_accuracy: 0.9797
Epoch 48/50
100/100 [==============================] - 9s 94ms/step - loss: 1.6636 - binary_crossentropy: 0.1267 - jaccard_coef_int: 0.2348 - binary_accuracy: 0.9895 - val_loss: 3.2881 - val_binary_crossentropy: 0.2501 - val_jaccard_coef_int: 0.0523 - val_binary_accuracy: 0.9790
Epoch 49/50
100/100 [==============================] - 9s 94ms/step - loss: 1.6572 - binary_crossentropy: 0.1310 - jaccard_coef_int: 0.2370 - binary_accuracy: 0.9892 - val_loss: 3.2684 - val_binary_crossentropy: 0.2591 - val_jaccard_coef_int: 0.0541 - val_binary_accuracy: 0.9783
Epoch 50/50
100/100 [==============================] - 9s 92ms/step - loss: 1.5684 - binary_crossentropy: 0.1279 - jaccard_coef_int: 0.2560 - binary_accuracy: 0.9895 - val_

...

Epoch 1/50
100/100 [==============================] - 42s 422ms/step - loss: 4.9473 - binary_crossentropy: 0.1796 - jaccard_coef_int: 0.0180 - binary_accuracy: 0.9437 - val_loss: 4.3050 - val_binary_crossentropy: 0.1572 - val_jaccard_coef_int: 0.0181 - val_binary_accuracy: 0.9685
Epoch 2/50
100/100 [==============================] - 10s 96ms/step - loss: 3.4341 - binary_crossentropy: 0.1604 - jaccard_coef_int: 0.1060 - binary_accuracy: 0.9501 - val_loss: 4.7449 - val_binary_crossentropy: 0.1651 - val_jaccard_coef_int: 0.0127 - val_binary_accuracy: 0.9699
Epoch 3/50
100/100 [==============================] - 10s 96ms/step - loss: 2.9827 - binary_crossentropy: 0.1822 - jaccard_coef_int: 0.1300 - binary_accuracy: 0.9528 - val_loss: 3.3964 - val_binary_crossentropy: 0.1615 - val_jaccard_coef_int: 0.0591 - val_binary_accuracy: 0.9681
Epoch 4/50
100/100 [==============================] - 9s 95ms/step - loss: 2.8230 - binary_crossentropy: 0.2044 - jaccard_coef_int: 0.1401 - binary_accuracy: 0

Epoch 7/50
100/100 [==============================] - 12s 118ms/step - loss: 2.1169 - binary_crossentropy: 0.1967 - jaccard_coef_int: 0.2168 - binary_accuracy: 0.9673 - val_loss: 2.0955 - val_binary_crossentropy: 0.1175 - val_jaccard_coef_int: 0.1815 - val_binary_accuracy: 0.9795
Epoch 8/50
100/100 [==============================] - 12s 119ms/step - loss: 2.0427 - binary_crossentropy: 0.2014 - jaccard_coef_int: 0.2233 - binary_accuracy: 0.9688 - val_loss: 3.6797 - val_binary_crossentropy: 0.5832 - val_jaccard_coef_int: 0.1070 - val_binary_accuracy: 0.9127
Epoch 9/50
100/100 [==============================] - 12s 119ms/step - loss: 2.0124 - binary_crossentropy: 0.2118 - jaccard_coef_int: 0.2311 - binary_accuracy: 0.9685 - val_loss: 2.3259 - val_binary_crossentropy: 0.2407 - val_jaccard_coef_int: 0.1767 - val_binary_accuracy: 0.9651
Epoch 10/50
100/100 [==============================] - 12s 120ms/step - loss: 2.0945 - binary_crossentropy: 0.2238 - jaccard_coef_int: 0.2190 - binary_accura

Epoch 13/50
100/100 [==============================] - 12s 118ms/step - loss: 2.7705 - binary_crossentropy: 0.9742 - jaccard_coef_int: 0.6233 - binary_accuracy: 0.8570 - val_loss: 2.7552 - val_binary_crossentropy: 0.9784 - val_jaccard_coef_int: 0.6457 - val_binary_accuracy: 0.8558
Epoch 14/50
100/100 [==============================] - 12s 118ms/step - loss: 2.7309 - binary_crossentropy: 0.9839 - jaccard_coef_int: 0.6302 - binary_accuracy: 0.8588 - val_loss: 2.8120 - val_binary_crossentropy: 0.9903 - val_jaccard_coef_int: 0.6411 - val_binary_accuracy: 0.8527
Epoch 15/50
100/100 [==============================] - 12s 119ms/step - loss: 2.7006 - binary_crossentropy: 1.0015 - jaccard_coef_int: 0.6346 - binary_accuracy: 0.8602 - val_loss: 2.8088 - val_binary_crossentropy: 1.0630 - val_jaccard_coef_int: 0.6366 - val_binary_accuracy: 0.8531
Epoch 16/50
100/100 [==============================] - 12s 121ms/step - loss: 2.6408 - binary_crossentropy: 0.9965 - jaccard_coef_int: 0.6393 - binary_acc

Epoch 19/50
100/100 [==============================] - 12s 119ms/step - loss: 0.5683 - binary_crossentropy: 0.0617 - jaccard_coef_int: 0.6684 - binary_accuracy: 0.9945 - val_loss: 1.0784 - val_binary_crossentropy: 0.1670 - val_jaccard_coef_int: 0.4250 - val_binary_accuracy: 0.9863
Epoch 20/50
100/100 [==============================] - 12s 118ms/step - loss: 0.6445 - binary_crossentropy: 0.0696 - jaccard_coef_int: 0.6529 - binary_accuracy: 0.9935 - val_loss: 1.0778 - val_binary_crossentropy: 0.1643 - val_jaccard_coef_int: 0.4269 - val_binary_accuracy: 0.9861
Epoch 21/50
100/100 [==============================] - 12s 118ms/step - loss: 0.6170 - binary_crossentropy: 0.0667 - jaccard_coef_int: 0.6577 - binary_accuracy: 0.9937 - val_loss: 1.0784 - val_binary_crossentropy: 0.1555 - val_jaccard_coef_int: 0.4273 - val_binary_accuracy: 0.9860
Epoch 22/50
100/100 [==============================] - 12s 118ms/step - loss: 0.5743 - binary_crossentropy: 0.0646 - jaccard_coef_int: 0.6665 - binary_acc

Epoch 25/50
100/100 [==============================] - 9s 95ms/step - loss: 3.9626 - binary_crossentropy: 0.1988 - jaccard_coef_int: 0.0419 - binary_accuracy: 0.9798 - val_loss: 3.0854 - val_binary_crossentropy: 0.4269 - val_jaccard_coef_int: 0.0778 - val_binary_accuracy: 0.9681
Epoch 26/50
100/100 [==============================] - 10s 95ms/step - loss: 4.1913 - binary_crossentropy: 0.3474 - jaccard_coef_int: 0.0365 - binary_accuracy: 0.9712 - val_loss: 3.3646 - val_binary_crossentropy: 0.4150 - val_jaccard_coef_int: 0.0739 - val_binary_accuracy: 0.9657
Epoch 27/50
100/100 [==============================] - 9s 95ms/step - loss: 4.0624 - binary_crossentropy: 0.3076 - jaccard_coef_int: 0.0385 - binary_accuracy: 0.9718 - val_loss: 3.1473 - val_binary_crossentropy: 0.3617 - val_jaccard_coef_int: 0.0769 - val_binary_accuracy: 0.9675
Epoch 28/50
100/100 [==============================] - 9s 95ms/step - loss: 3.5772 - binary_crossentropy: 0.1103 - jaccard_coef_int: 0.0524 - binary_accuracy: 

In [13]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/depthtest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, epochs=50, callbacks=[<keras.ca..., steps_per_epoch=100, validation_data=(array([[[...)`


Epoch 1/50
100/100 [==============================] - 19s 193ms/step - loss: 1.7660 - binary_crossentropy: 0.0506 - jaccard_coef_int: 0.6985 - binary_accuracy: 0.9889 - val_loss: 0.6424 - val_binary_crossentropy: 0.0441 - val_jaccard_coef_int: 0.6359 - val_binary_accuracy: 0.9905
Epoch 2/50
100/100 [==============================] - 12s 125ms/step - loss: 0.7048 - binary_crossentropy: 0.0212 - jaccard_coef_int: 0.7369 - binary_accuracy: 0.9954 - val_loss: 0.6691 - val_binary_crossentropy: 0.0467 - val_jaccard_coef_int: 0.6443 - val_binary_accuracy: 0.9908
Epoch 3/50
100/100 [==============================] - 13s 131ms/step - loss: 0.6704 - binary_crossentropy: 0.0253 - jaccard_coef_int: 0.7533 - binary_accuracy: 0.9941 - val_loss: 0.6443 - val_binary_crossentropy: 0.0901 - val_jaccard_coef_int: 0.6178 - val_binary_accuracy: 0.9902
Epoch 4/50
100/100 [==============================] - 12s 125ms/step - loss: 0.4583 - binary_crossentropy: 0.0233 - jaccard_coef_int: 0.7591 - binary_accurac

100/100 [==============================] - 11s 111ms/step - loss: 0.3666 - binary_crossentropy: 0.0395 - jaccard_coef_int: 0.7544 - binary_accuracy: 0.9957 - val_loss: 0.5732 - val_binary_crossentropy: 0.0974 - val_jaccard_coef_int: 0.6519 - val_binary_accuracy: 0.9909
Epoch 31/50
100/100 [==============================] - 11s 114ms/step - loss: 0.3513 - binary_crossentropy: 0.0389 - jaccard_coef_int: 0.7659 - binary_accuracy: 0.9954 - val_loss: 0.5876 - val_binary_crossentropy: 0.0961 - val_jaccard_coef_int: 0.6445 - val_binary_accuracy: 0.9908
Epoch 32/50
100/100 [==============================] - 11s 110ms/step - loss: 0.3428 - binary_crossentropy: 0.0409 - jaccard_coef_int: 0.7695 - binary_accuracy: 0.9954 - val_loss: 0.5690 - val_binary_crossentropy: 0.1014 - val_jaccard_coef_int: 0.6553 - val_binary_accuracy: 0.9910
Epoch 33/50
100/100 [==============================] - 11s 113ms/step - loss: 0.3587 - binary_crossentropy: 0.0433 - jaccard_coef_int: 0.7588 - binary_accuracy: 0.995

Epoch 7/50
100/100 [==============================] - 9s 89ms/step - loss: 4.6431 - binary_crossentropy: 1.3256 - jaccard_coef_int: 0.2917 - binary_accuracy: 0.7880 - val_loss: 5.4902 - val_binary_crossentropy: 1.4305 - val_jaccard_coef_int: 0.1023 - val_binary_accuracy: 0.7992
Epoch 8/50
100/100 [==============================] - 9s 90ms/step - loss: 4.6364 - binary_crossentropy: 1.4424 - jaccard_coef_int: 0.2988 - binary_accuracy: 0.7866 - val_loss: 4.8290 - val_binary_crossentropy: 1.4820 - val_jaccard_coef_int: 0.2238 - val_binary_accuracy: 0.7920
Epoch 9/50
100/100 [==============================] - 9s 90ms/step - loss: 4.5929 - binary_crossentropy: 1.4987 - jaccard_coef_int: 0.2993 - binary_accuracy: 0.7891 - val_loss: 4.9282 - val_binary_crossentropy: 1.5515 - val_jaccard_coef_int: 0.2099 - val_binary_accuracy: 0.7898
Epoch 10/50
100/100 [==============================] - 9s 90ms/step - loss: 4.5198 - binary_crossentropy: 1.5128 - jaccard_coef_int: 0.3056 - binary_accuracy: 0.79

...

Epoch 20/50
100/100 [==============================] - 11s 113ms/step - loss: 2.2558 - binary_crossentropy: 0.5660 - jaccard_coef_int: 0.2943 - binary_accuracy: 0.9361 - val_loss: 3.3671 - val_binary_crossentropy: 0.8758 - val_jaccard_coef_int: 0.1792 - val_binary_accuracy: 0.8972
Epoch 21/50
100/100 [==============================] - 11s 114ms/step - loss: 2.2542 - binary_crossentropy: 0.5816 - jaccard_coef_int: 0.2977 - binary_accuracy: 0.9355 - val_loss: 3.5175 - val_binary_crossentropy: 0.8427 - val_jaccard_coef_int: 0.1299 - val_binary_accuracy: 0.9076
Epoch 22/50
100/100 [==============================] - 11s 113ms/step - loss: 2.1334 - binary_crossentropy: 0.5522 - jaccard_coef_int: 0.3147 - binary_accuracy: 0.9396 - val_loss: 3.3692 - val_binary_crossentropy: 0.9277 - val_jaccard_coef_int: 0.1805 - val_binary_accuracy: 0.8966
Epoch 23/50
100/100 [==============================] - 11s 112ms/step - loss: 2.0966 - binary_crossentropy: 0.5659 - jaccard_coef_int: 0.3302 - binary_acc

100/100 [==============================] - 11s 114ms/step - loss: 1.5441 - binary_crossentropy: 0.5332 - jaccard_coef_int: 0.4523 - binary_accuracy: 0.9533 - val_loss: 3.6208 - val_binary_crossentropy: 1.2094 - val_jaccard_coef_int: 0.1403 - val_binary_accuracy: 0.8965
Epoch 50/50
100/100 [==============================] - 12s 115ms/step - loss: 1.5260 - binary_crossentropy: 0.5382 - jaccard_coef_int: 0.4595 - binary_accuracy: 0.9534 - val_loss: 3.4894 - val_binary_crossentropy: 1.2141 - val_jaccard_coef_int: 0.1622 - val_binary_accuracy: 0.8957
0.895508489884393
             precision    recall  f1-score   support

        0.0       0.93      0.96      0.94   1014360
        1.0       0.32      0.21      0.25     92840

avg / total       0.88      0.90      0.89   1107200



label: 82
{82: 1.0}
composition with label != 0
Y_train 0.04508856682769726 0.008267914653784219
composition with label != 0
Y_val 0.0 0.0
composition with label != 0
Y_test 0.046242774566473986 0.0079407514450867

Epoch 26/50
100/100 [==============================] - 11s 115ms/step - loss: 2.8013 - binary_crossentropy: 0.1428 - jaccard_coef_int: 0.1466 - binary_accuracy: 0.9848 - val_loss: 28.8871 - val_binary_crossentropy: 0.0010 - val_jaccard_coef_int: 1.0000 - val_binary_accuracy: 1.0000
Epoch 27/50
100/100 [==============================] - 11s 113ms/step - loss: 2.8764 - binary_crossentropy: 0.0995 - jaccard_coef_int: 0.1586 - binary_accuracy: 0.9886 - val_loss: 49.7593 - val_binary_crossentropy: 10.2550 - val_jaccard_coef_int: 2.1829e-16 - val_binary_accuracy: 0.1472
Epoch 28/50
100/100 [==============================] - 11s 114ms/step - loss: 3.3264 - binary_crossentropy: 0.0987 - jaccard_coef_int: 0.1447 - binary_accuracy: 0.9889 - val_loss: 28.5431 - val_binary_crossentropy: 0.0011 - val_jaccard_coef_int: 1.0000 - val_binary_accuracy: 1.0000
Epoch 29/50
100/100 [==============================] - 11s 112ms/step - loss: 2.6336 - binary_crossentropy: 0.0974 - jaccard_coef_int: 0.1494 - bi

100/100 [==============================] - 9s 90ms/step - loss: 3.8104 - binary_crossentropy: 0.0574 - jaccard_coef_int: 0.0618 - binary_accuracy: 0.9834 - val_loss: 3.6956 - val_binary_crossentropy: 0.0662 - val_jaccard_coef_int: 0.0325 - val_binary_accuracy: 0.9876
Epoch 3/50
100/100 [==============================] - 9s 89ms/step - loss: 3.2019 - binary_crossentropy: 0.0492 - jaccard_coef_int: 0.0785 - binary_accuracy: 0.9881 - val_loss: 8.0518 - val_binary_crossentropy: 0.0184 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9975
Epoch 4/50
100/100 [==============================] - 9s 89ms/step - loss: 3.0870 - binary_crossentropy: 0.0450 - jaccard_coef_int: 0.0775 - binary_accuracy: 0.9906 - val_loss: 6.4254 - val_binary_crossentropy: 0.0310 - val_jaccard_coef_int: 7.9937e-14 - val_binary_accuracy: 0.9955
Epoch 5/50
100/100 [==============================] - 9s 92ms/step - loss: 3.0300 - binary_crossentropy: 0.0537 - jaccard_coef_int: 0.0791 - binary_accuracy: 0.9901 - val

100/100 [==============================] - 9s 91ms/step - loss: 1.8192 - binary_crossentropy: 0.0624 - jaccard_coef_int: 0.1960 - binary_accuracy: 0.9945 - val_loss: 3.5159 - val_binary_crossentropy: 0.2041 - val_jaccard_coef_int: 0.0391 - val_binary_accuracy: 0.9823
Epoch 32/50
100/100 [==============================] - 9s 90ms/step - loss: 1.7202 - binary_crossentropy: 0.0592 - jaccard_coef_int: 0.2175 - binary_accuracy: 0.9948 - val_loss: 3.4821 - val_binary_crossentropy: 0.1018 - val_jaccard_coef_int: 0.0348 - val_binary_accuracy: 0.9912
Epoch 33/50
100/100 [==============================] - 9s 90ms/step - loss: 1.7172 - binary_crossentropy: 0.0556 - jaccard_coef_int: 0.2158 - binary_accuracy: 0.9951 - val_loss: 10.5211 - val_binary_crossentropy: 0.0248 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9976
Epoch 34/50
100/100 [==============================] - 9s 91ms/step - loss: 1.8051 - binary_crossentropy: 0.0565 - jaccard_coef_int: 0.2103 - binary_accuracy: 0.9950 - val

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/depthtest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, epochs=50, callbacks=[<keras.ca..., steps_per_epoch=100, validation_data=(array([[[...)`


Epoch 1/50
100/100 [==============================] - 25s 249ms/step - loss: 3.8407 - binary_crossentropy: 0.2211 - jaccard_coef_int: 0.2703 - binary_accuracy: 0.9063 - val_loss: 3.6069 - val_binary_crossentropy: 0.2304 - val_jaccard_coef_int: 0.2017 - val_binary_accuracy: 0.9167
Epoch 2/50
100/100 [==============================] - 12s 116ms/step - loss: 2.3493 - binary_crossentropy: 0.1933 - jaccard_coef_int: 0.4686 - binary_accuracy: 0.9294 - val_loss: 2.9270 - val_binary_crossentropy: 0.2702 - val_jaccard_coef_int: 0.2482 - val_binary_accuracy: 0.9190
Epoch 3/50
100/100 [==============================] - 11s 115ms/step - loss: 1.9812 - binary_crossentropy: 0.2031 - jaccard_coef_int: 0.5048 - binary_accuracy: 0.9348 - val_loss: 2.0838 - val_binary_crossentropy: 0.2203 - val_jaccard_coef_int: 0.4691 - val_binary_accuracy: 0.9313
Epoch 4/50
100/100 [==============================] - 12s 116ms/step - loss: 2.0035 - binary_crossentropy: 0.2300 - jaccard_coef_int: 0.4876 - binary_accurac

100/100 [==============================] - 12s 116ms/step - loss: 0.6238 - binary_crossentropy: 0.1323 - jaccard_coef_int: 0.7855 - binary_accuracy: 0.9771 - val_loss: 0.5492 - val_binary_crossentropy: 0.1109 - val_jaccard_coef_int: 0.8064 - val_binary_accuracy: 0.9802
Epoch 31/50
100/100 [==============================] - 12s 116ms/step - loss: 0.5910 - binary_crossentropy: 0.1221 - jaccard_coef_int: 0.7942 - binary_accuracy: 0.9784 - val_loss: 1.0033 - val_binary_crossentropy: 0.2822 - val_jaccard_coef_int: 0.6833 - val_binary_accuracy: 0.9618
Epoch 32/50
100/100 [==============================] - 11s 115ms/step - loss: 0.5830 - binary_crossentropy: 0.1213 - jaccard_coef_int: 0.7957 - binary_accuracy: 0.9787 - val_loss: 0.4620 - val_binary_crossentropy: 0.0870 - val_jaccard_coef_int: 0.8311 - val_binary_accuracy: 0.9838
Epoch 33/50
100/100 [==============================] - 11s 115ms/step - loss: 0.5526 - binary_crossentropy: 0.1138 - jaccard_coef_int: 0.8037 - binary_accuracy: 0.980

Epoch 7/50
100/100 [==============================] - 12s 116ms/step - loss: 1.4289 - binary_crossentropy: 0.1364 - jaccard_coef_int: 0.4801 - binary_accuracy: 0.9632 - val_loss: 1.3863 - val_binary_crossentropy: 0.1445 - val_jaccard_coef_int: 0.5216 - val_binary_accuracy: 0.9607
Epoch 8/50
100/100 [==============================] - 12s 116ms/step - loss: 1.3150 - binary_crossentropy: 0.1343 - jaccard_coef_int: 0.5075 - binary_accuracy: 0.9655 - val_loss: 1.2057 - val_binary_crossentropy: 0.1222 - val_jaccard_coef_int: 0.5695 - val_binary_accuracy: 0.9658
Epoch 9/50
100/100 [==============================] - 12s 117ms/step - loss: 1.2085 - binary_crossentropy: 0.1292 - jaccard_coef_int: 0.5347 - binary_accuracy: 0.9680 - val_loss: 1.6484 - val_binary_crossentropy: 0.2173 - val_jaccard_coef_int: 0.4173 - val_binary_accuracy: 0.9539
Epoch 10/50
100/100 [==============================] - 12s 116ms/step - loss: 1.1256 - binary_crossentropy: 0.1250 - jaccard_coef_int: 0.5570 - binary_accura

...

Epoch 10/50
100/100 [==============================] - 9s 94ms/step - loss: 1.3455 - binary_crossentropy: 0.2038 - jaccard_coef_int: 0.5697 - binary_accuracy: 0.9548 - val_loss: 1.0700 - val_binary_crossentropy: 0.1413 - val_jaccard_coef_int: 0.6310 - val_binary_accuracy: 0.9658
Epoch 11/50
100/100 [==============================] - 9s 95ms/step - loss: 1.3118 - binary_crossentropy: 0.2007 - jaccard_coef_int: 0.5759 - binary_accuracy: 0.9561 - val_loss: 1.0930 - val_binary_crossentropy: 0.1444 - val_jaccard_coef_int: 0.6248 - val_binary_accuracy: 0.9645
Epoch 12/50
100/100 [==============================] - 10s 95ms/step - loss: 1.2777 - binary_crossentropy: 0.1952 - jaccard_coef_int: 0.5834 - binary_accuracy: 0.9571 - val_loss: 1.0856 - val_binary_crossentropy: 0.1408 - val_jaccard_coef_int: 0.6271 - val_binary_accuracy: 0.9654
Epoch 13/50
100/100 [==============================] - 9s 94ms/step - loss: 1.2382 - binary_crossentropy: 0.1919 - jaccard_coef_int: 0.5978 - binary_accuracy: 

100/100 [==============================] - 9s 94ms/step - loss: 0.6071 - binary_crossentropy: 0.1127 - jaccard_coef_int: 0.7775 - binary_accuracy: 0.9788 - val_loss: 0.4331 - val_binary_crossentropy: 0.0599 - val_jaccard_coef_int: 0.8282 - val_binary_accuracy: 0.9861
Epoch 40/50
100/100 [==============================] - 9s 93ms/step - loss: 0.5989 - binary_crossentropy: 0.1119 - jaccard_coef_int: 0.7786 - binary_accuracy: 0.9793 - val_loss: 0.4651 - val_binary_crossentropy: 0.0689 - val_jaccard_coef_int: 0.8157 - val_binary_accuracy: 0.9848
Epoch 41/50
100/100 [==============================] - 9s 94ms/step - loss: 0.5784 - binary_crossentropy: 0.1082 - jaccard_coef_int: 0.7847 - binary_accuracy: 0.9800 - val_loss: 0.5758 - val_binary_crossentropy: 0.0957 - val_jaccard_coef_int: 0.7738 - val_binary_accuracy: 0.9807
Epoch 42/50
100/100 [==============================] - 9s 95ms/step - loss: 0.5726 - binary_crossentropy: 0.1080 - jaccard_coef_int: 0.7884 - binary_accuracy: 0.9800 - val_

Epoch 16/50
100/100 [==============================] - 10s 95ms/step - loss: 0.7509 - binary_crossentropy: 0.1222 - jaccard_coef_int: 0.6472 - binary_accuracy: 0.9811 - val_loss: 0.8193 - val_binary_crossentropy: 0.1138 - val_jaccard_coef_int: 0.5909 - val_binary_accuracy: 0.9827
Epoch 17/50
100/100 [==============================] - 10s 97ms/step - loss: 0.7420 - binary_crossentropy: 0.1243 - jaccard_coef_int: 0.6520 - binary_accuracy: 0.9811 - val_loss: 0.7058 - val_binary_crossentropy: 0.0847 - val_jaccard_coef_int: 0.6219 - val_binary_accuracy: 0.9865
Epoch 18/50
100/100 [==============================] - 10s 95ms/step - loss: 0.7600 - binary_crossentropy: 0.1300 - jaccard_coef_int: 0.6434 - binary_accuracy: 0.9809 - val_loss: 0.7681 - val_binary_crossentropy: 0.0951 - val_jaccard_coef_int: 0.5941 - val_binary_accuracy: 0.9852
Epoch 19/50
100/100 [==============================] - 10s 96ms/step - loss: 0.7230 - binary_crossentropy: 0.1255 - jaccard_coef_int: 0.6593 - binary_accurac

100/100 [==============================] - 10s 98ms/step - loss: 0.4925 - binary_crossentropy: 0.0949 - jaccard_coef_int: 0.7548 - binary_accuracy: 0.9873 - val_loss: 0.4729 - val_binary_crossentropy: 0.0662 - val_jaccard_coef_int: 0.7213 - val_binary_accuracy: 0.9910
Epoch 46/50
100/100 [==============================] - 10s 95ms/step - loss: 0.4369 - binary_crossentropy: 0.0793 - jaccard_coef_int: 0.7784 - binary_accuracy: 0.9888 - val_loss: 0.5890 - val_binary_crossentropy: 0.0845 - val_jaccard_coef_int: 0.6588 - val_binary_accuracy: 0.9895
Epoch 47/50
100/100 [==============================] - 10s 97ms/step - loss: 0.4404 - binary_crossentropy: 0.0816 - jaccard_coef_int: 0.7772 - binary_accuracy: 0.9886 - val_loss: 0.6231 - val_binary_crossentropy: 0.0932 - val_jaccard_coef_int: 0.6418 - val_binary_accuracy: 0.9888
Epoch 48/50
100/100 [==============================] - 10s 95ms/step - loss: 0.4224 - binary_crossentropy: 0.0766 - jaccard_coef_int: 0.7839 - binary_accuracy: 0.9892 - 

100/100 [==============================] - 9s 95ms/step - loss: 1.0203 - binary_crossentropy: 0.2473 - jaccard_coef_int: 0.6996 - binary_accuracy: 0.9597 - val_loss: 1.6945 - val_binary_crossentropy: 0.4988 - val_jaccard_coef_int: 0.5572 - val_binary_accuracy: 0.9315
Epoch 23/50
100/100 [==============================] - 9s 94ms/step - loss: 1.0357 - binary_crossentropy: 0.2600 - jaccard_coef_int: 0.6977 - binary_accuracy: 0.9589 - val_loss: 1.3096 - val_binary_crossentropy: 0.3607 - val_jaccard_coef_int: 0.6483 - val_binary_accuracy: 0.9461
Epoch 24/50
100/100 [==============================] - 9s 94ms/step - loss: 1.0259 - binary_crossentropy: 0.2632 - jaccard_coef_int: 0.7012 - binary_accuracy: 0.9592 - val_loss: 1.3461 - val_binary_crossentropy: 0.3792 - val_jaccard_coef_int: 0.6339 - val_binary_accuracy: 0.9448
Epoch 25/50
100/100 [==============================] - 9s 94ms/step - loss: 1.0018 - binary_crossentropy: 0.2614 - jaccard_coef_int: 0.7069 - binary_accuracy: 0.9601 - val_